In [13]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

### 1. PreProcessing

In [14]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),          
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

data_dir = 'flowers'

dataset = datasets.ImageFolder(data_dir, transform=transform)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_load = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_load = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Classes: {dataset.classes}")

Classes: ['dandelion', 'sunflower']


In [15]:
import torch
import torch.nn as nn
from torchvision import models


model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False


num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 256),
    nn.ReLU(),
    nn.Linear(256, 2)
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
num_epochs = 10

for epoch in  range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_load:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)


        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_load):.4f}')




model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in val_load:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

Epoch [1/10], Loss: 0.2566


KeyboardInterrupt: 